<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;"> 
    <img src="https://upload.wikimedia.org/wikipedia/fa/thumb/f/fd/University_of_Tehran_logo.svg/1200px-University_of_Tehran_logo.svg.png" style="float:left;width:140px;padding-top: 10px" />
    <div style="font-size: 20px;padding-top: 30px;"><b>پروژه یک : سرچ</b></div>
    <div style="font-size: 16px;margin-top: 20px;">ثمین مهدی زاده 810196623</div>
</div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6"> 
دراین پروژه با استفاده از الگوریتم های جست وجوی آگاهانه ونا آگاهانه راه حل مناسبی برای یک مسئله پیدا شده است و سپس این الگوریتم ها از نظر زمان اجرا،تعداد استیت های دیده شده و... با هم مقایسه شده اند.
    <br/>
مسئله به این صورت تعریف می شود که در ابتدا یک مار با طول یک به همراه تعدادی دانه با امتیاز یک یا دو در صفحه وجود دارند و طول مار هم بعد از هر بار خوردن دانه زیاد می شود.در هر حرکت اگر مار با بدن خود تماس پیدا کند از بازی خارج می شود.هدف این است که مار در بهینه ترین حالت ممکن همه دانه ها را بخورد.
</div>    

In [1]:
from collections import deque  
import time
import heapq
import pandas as pd
from IPython.display import HTML 

In [2]:
file = open('./tests/test3.txt', 'r') 
lines = file.readlines() 
count=0
seeds = {}  
for line in lines: 
    l = line.replace('\n','').split(",")
    if(count == 0):
        boundry = (int(l[0]),int(l[1]))
    if(count == 1):
        snake = [(int(l[0]),int(l[1]))]
    if(count>2):
        seeds[(int(l[0]),int(l[1]))] = int(l[2]) 
    count+=1

<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;"> 
    <b>مدل کردن مسئله </b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
برای مدل کردن مسئله کلاس Node پیاده سازی شده است.
<br/>
    این کلاس دارای دو فیلد snakeوseedsاست که در واقع بیانگرstate مسئله هستند.snake لیستی است که شامل مختصات هر نقطه از بدن مار است(سر مار اولین خانه لیست و دم مار آخرین خانه آن است).seeds که همان دانه ها هستند به صورت یک dictionary پیاده سازی شده است که کلید آن مختصات هر دانه و مقدار آن امتیاز دانه در آن مرحله است.
    <br/>
    actions شامل تمام حرکات تعریف شده برای مار است که در این جا بالا،راست،پایین وچپ می توانند باشند.
    <br/>
    level بیان کننده ی هزینه ی مسیر است و بیان می کند هر نود در چه سطحی دیده شده است.
    <br/>
    path نیز نشان می دهد که برای رسیدن به این مرحله چه حرکاتی انجام شده است.
    <br/>
    boundry   سایز صفحه بازی را برای حرکت مار مشخص می کند.
    <br/>
    در انتها نیز eaten_food مشخص می کند که آیا در مرحله قبل دانه ای خورده شده است یا نه تا با توجه به آن طول بدن را تغییر دهد.
    <br/>
    از آن جایی که بازی وقتی تمام می شود که همه ی دانه ها خورده شده باشند goal state استیتی است که در آن seeds خالی باشد.تابعis_goal با بررسی این موضوع مشخص می کند که به هدف نهایی رسیده ایم یا خیر.در صورتی که در یک مرحله باشیم باید بتوانیم حرکات مجاز برای مرحله بعد را مشخص کنیم و به کمک آن حالت های ممکن موجود برای مرحله بعد را به دست آوریم.تابع get_child با بررسی حرکات،حالت های مجاز برای مرحله ی بعد را به ما بر می گرداند.
برای این که مشخص کنیم که به مرحله تکراری رسیده ایم یا خیر باید برای هر مرحله کلید مجزایی داشته باشیم.می دانیم که اگر استیت دو مرحله یکسان باشد بیانگر این است که مشابه هم هستند  بنابراین کلید هر مرحله را رشته ای از stateکه همان snakeوseeds می شوند در نظر می گیریم.



    
</div>

In [3]:
class Node:
    
    def __init__(self,snake,seeds,boundry,level = 0,path ="",eaten_seed = False):
        self.snake = snake
        self.seeds = seeds
        self.actions = ['U','R','D','L']
        self.level = level
        self.path = path
        self.boundry = boundry
        self.eaten_seed = eaten_seed
        
    def get_childs(self):
        childs = []
        
        for action in self.actions:
            eat = False
            collision = False
            new_snake = self.snake.copy()
            
            if(action == 'U'):
                if(self.snake[0][0]==0):
                    head = (self.boundry[0]-1,self.snake[0][1])
                else:
                    head = (self.snake[0][0]-1,self.snake[0][1])
                    
            elif(action == 'D'):
                if(self.snake[0][0]==self.boundry[0]-1):
                    head = (0,self.snake[0][1])
                else:
                    head = (self.snake[0][0]+1,self.snake[0][1])
            elif(action == 'L'):
                if(self.snake[0][1]==0):
                    head = (self.snake[0][0],self.boundry[1]-1)
                else:
                    head = (self.snake[0][0],self.snake[0][1]-1)
            
            if(action == 'R'):
                if(self.snake[0][1]==self.boundry[1]-1):
                    head = (self.snake[0][0],0)
                else:
                    head = (self.snake[0][0],self.snake[0][1]+1)
                    
            new_snake.insert(0,head)
            if(not self.eaten_seed):
                new_snake.pop()
      
            if((new_snake[0] in new_snake[1:]) or (len(new_snake)==2 and new_snake[0]==self.snake[-1])):
                collision = True
                
            if(not collision):
                new_seeds = self.seeds.copy()
                if(new_snake[0] in self.seeds):
                    eat = True
                    if(new_seeds[new_snake[0]]==1):
                        del new_seeds[new_snake[0]]
                    else:
                        new_seeds[new_snake[0]]-=1
                child = Node(new_snake,new_seeds,self.boundry,self.level+1,self.path+action,eat)
                childs.append(child)
        return childs
    
    def is_goal(self):
        return not self.seeds
    
    def get_key(self):
        return str((self.snake,self.seeds))

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
برای تعریف initial_state حالت اولیه از فایل خوانده می شود و به کمک آن مقادیر seeds,snake,boundry مقدار دهی می شوند.سپس به کمک کلاس Node مرحله ابتدایی بازی ساخته می شود.
</div>

In [4]:
initial_state = Node(snake,seeds,boundry)

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    در ادامه روش پیاده سازی الگوریتم های استفاده شده آورده شده است.
</div>

<div style="direction:rtl;font-size: 20px;text-align: justify;line-height: 1.6;"> 
    <b>BFS</b>
</div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    در این روش نود ها از کمترین سطح دیده می شوند و پس از بررسی همه ی نود های موجود در یک سطح اگر به نود هدف نرسیده باشیم یک سطح به پایین می رویم.از آن جا که این الگوریتم از کمترین سطح شروع می کند اولین نودی که شرط goal را داشته باشد قطعا نود با کمترین هزینه است.
    <br/>
    برای پیاده سازی این الگوریتم کلاس BFS_serach زده شده است.این کلاس هنگام فراخوانی state اولیه را می گیرد.سپس با صدا زدن start_BFS سرچ آغاز می شود.برای پیاده سازی این تابع از دو متغیر frontier و explored استفاده شده است.
    <br/>
frontier  با استفاده از یک queue دو طرفه زده شده است(queue دو طرفه است تا در IDS هم بتوان از کد مشابه استفاده کرد). ازfrontier برای نگه داری نود هایی که باید گسترس داده بشوند استفاده شده است.  
    <br/>
پیاده سازی explored با استفاده ازset انجام شده است چرا که هزینه ی جست و جو و اضافه کردن نود در آن  کم است.explored شامل کلید نود هایی است که تا به حال گسترس داده شده اند.
    <br/>
در هر مرحله چپ ترین نود موجود درfrontier برداشته می شود و بچه های آن بررسی می شوند(خود نود در explored قرار می گیرد).در صورتی که هر کدام از بچه ها قبلا گسترش داده شده بودند و یا در frontier موجود بودند دوباره لازم نیست که بررسی شوند بنابراین در frontier قرار نمی گیرند.در frontier خود نود ها موجود اند و بررسی این که نود جدیدی در آن نباشد هزینه بر است. از آن جایی که نود در هیچ کدام ازfrontier یا  explored نباید وجود داشته باشد آن ها را باهم ادغام کرده و برای بررسی این موضوع کلید نود هایی که در frontier قرار می گیرند نیز در explored ذخیره می کنیم.
    <br/>
    به ازای هر بچه که به آن رسیدیم بررسی می کنیم که اگر شرط goal را پاس کرده بود آن را به عنوان جواب بر می گردانیم.
    
</div>

In [5]:
class BFS_search:
    
    def __init__(self,initial_state):
        self.initial_state = initial_state
    
    def start_BFS(self):
        node = self.initial_state
        total_state = 1
        total_distinct_state = 1
        if node.is_goal():
            return node.path,total_state,total_distinct_state 
        frontier = deque([node])
        explored = set([node.get_key])
        while True:
            if len(frontier) == 0:
                return "",self.total_state,self.total_distinct_state 
            node = frontier.popleft()
            childs = node.get_childs()
            for child in childs:
                total_state += 1
                if child.get_key() not in explored:
                    total_distinct_state +=1
                    if child.is_goal():
                        return child.path,total_state,total_distinct_state 
                    frontier.append(child)
                    explored.add(child.get_key())
                

In [6]:
start_time = time.time()
path, total_state, total_distinct_state  = BFS_search(initial_state).start_BFS()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ",total_state )
print("Distinct Nodes : ", total_distinct_state)

Solution's time : 4.31637978553772 seconds
Solution's Path :  URRDDDRRDRRRDDRRULLDLUULL
Solution's depth :  25
Nodes  :  434506
Distinct Nodes :  195025


<div style="direction:rtl;font-size:18px;"><b>IDS</b></div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
الگوریتم IDS در واقع ترکیبی از BFSوDFSاست.دراین الگوریتم ابتدا جست و جو را از کمترین سطح آغاز می کنیم و در صورت پیدا نکردن هدف سطح جست و جو را افزایش می دهیم.در این روش برای پیدا کردن حالت دلخواه الگوریتم DFS را در هر سطح انجام می دهیم.نکته ای که در این الگوریتم وجود دارد این است که برای پیدا کردن جواب بهینه لازم است تا نود های تکراری هم بررسی شوند چرا که ممکن است در شاخه ی دیگر همان نود در سطح پایین تری وجود داشته باشد و چک نکردن آن باعث می شود که به جواب بهینه نرسیم از این رو شرط لازم برای قرار دادن نود در frontier این است که در explored نباشد یا اگر بود این بار در سطح کمتری وجود داشته باشد.
<br/>
    برای پیاده سازی این الگوریتم از کلاس IDS_search استفاده شده است.این کلاس در مقدار دهی اولیه استیت شروع را دریافت می کند.مقادیر total_state و total_distinct_state هم به عنوان فیلد های این کلاس دخیره شده اند تا پس از هر بار تکرار مقادیر به دست آمده برای آن سطح به مقادیر قبلی اضافه شوند.total_visited_node نیز شامل کلید تمام نود هایی است که تا به اینجای کار دیده شده اند و از آن جهت شمارش نود های مجزا استفاده شده است.
    <br/>
   بعد از صدا زدن تابع start_IDS جست و جوی عمیق تا سطح مشخص شده توسط تابع DLS انجام می شود و در صورت پیدا نشدن،این تابع با سطح بالاتری صدا زده می شود تا بالاخره یکی از نود ها شرط goal را پاس کند و جواب را برگرداند.
</div>


In [7]:
class IDS_search:
    
    def __init__(self,initial_state):
        self.initial_state = initial_state
        self.total_state = 1
        self.total_distinct_state = 1
        self.total_visited_node = set([initial_state.get_key()])
        
    def DLS(self,level):
        node = self.initial_state
        if node.is_goal():
            return node.path,self.total_state,self.total_distinct_state 
        frontier = deque([node])
        explored = {}
        explored[node.get_key()] = node.level
        while True:
            if(len(frontier) == 0):
                return "",self.total_state,self.total_distinct_state 
            node = frontier.pop()
            if(node.level > level):
                continue;
            childs = node.get_childs()
            for child in childs:
                self.total_state+=1
                if(child.get_key() not in explored or explored[child.get_key()]>child.level):
                    if(child.get_key() not in self.total_visited_node):
                        self.total_visited_node.add(child.get_key())
                        self.total_distinct_state +=1
                    if child.is_goal():
                        return child.path,self.total_state,self.total_distinct_state 
                    frontier.append(child)
                    explored[child.get_key()] = child.level
                    
                 
    def start_IDS(self):
        level = 0 
        while True:
            path, nodes_count, distint_nodes_count = self.DLS(level)
            if(path != ""):
                break;
            level+=1
        return path, nodes_count, distint_nodes_count

In [8]:
start_time = time.time()
path, total_state, total_distinct_state  = IDS_search(initial_state).start_IDS()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ",total_state )
print("Distinct Nodes : ", total_distinct_state)

Solution's time : 52.764408826828 seconds
Solution's Path :  RUDDDRDRRDDRRRRRULLDLLLUU
Solution's depth :  25
Nodes  :  3831299
Distinct Nodes :  216618


<div style="direction:rtl;font-size:18px;"><b>A start/Weighted A start</b></div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    الگوریتم *A همانند BFS است با این تفاوت که در هر مرحله به جای برداشتن چپ ترین نود از fronier نودی برداشته می شود که داری اولویت بالاتری باشد.
    <br/>
    برای مشخص کردن این اولویت از جمع دوتابع g و h استفاده می کنیم.تابع g بیان کننده هزینه ی انجام شده برای رسیدن از نود اولیه به نود مورد نظر است و h بیان کننده پیش بینی ما برای برای رسیدن از این نود به نود هدف است.به تابع heuristic،h گفته می شود.یک heuristic در صورتی admissible است که مقدارش برای هر نود کمتر از هزینه ی واقعی برای رسیدن از آن نود به نود هدف باشد.همچنین  heuristic consistent است اگر اختلاف heuristic های بین هر دو نود کمتر از هزینه ی واقعی برای رسیدن از یکی از آن نود ها به نود دیگر باشد.
    <br/>
    الگوریتم *Weighted A همانند الگوریتم *A  است با این تفاوت که heuristic به دست آمده در عدد ثابتی ضرب می شود.
    <br/>
    همان طور که گفته شد در این الگوریتم نود ها براساس اولیت در صف قرار می گیرند به این منظور کلاس PriorityQueue پیاده سازی شده است تا با دریافت اولویت آن ها را در صف قرار دهد.
</div>

In [9]:
class PriorityQueue:
    
    def __init__(self):
        self._queue = []
        self._index = 0
        
    def is_empty(self):
        return not self._queue
    
    def push(self, item, priority):
        heapq.heappush(self._queue, (priority, self._index, item))
        self._index += 1
        
    def pop(self):
        return heapq.heappop(self._queue)[-1]

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    تابع زیر اولین heuristic پیاده سازی شده است.این heuristic جمع امتیازات دانه های هر مرحله را بر می گرداند.این تابع adimissible است چرا که حداقل حرکات برای رسیدن به هدف این است که همه ی دانه ها خورده شوند و در بهترین حالت هر دانه بعد از تنها یک حرکت خورده می شود.بنابراین تابع زیر قطعا همیشه هزینه کمتر یا مساوی هزینه ی واقعی را بر می گرداند.
    <br/>
    این تابع consistent نیز هست چرا که اگر بین هر دو مرحله دانه ای خورده شده نباشد،اختلاف heuristic ها برابر0  است که از هزینه ی واقعی کمتر است.همچنین اگر تعدادی دانه بین این دو مرحله خورده شده باشد حداقل حرکات برابر دانه های خورده شده است که همان اخلاف heuristic های دو مرحله است.
</div>

In [10]:
def calculate_first_heuristic(node,alpha):
    return alpha*sum(node.seeds.values())

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    دومین تابع heuristic نزدیک ترین فاصله ی افقی و عمودی از سر مار را برای هر دانه را حساب می کند تا با جمع آن ها کمترین تعداد حرکات برای رسیدن به آن دانه به دست بیاید. سپس بیش ترین آن ها را بر می گرداند.این الگوریتم admissible است چرا که برای خوردن همه ی دانه ها لازم است دانه با بیشترین فاصله هم خورده شود پس قطعا هزینه ی واقعی برای خوردن همه ی دانه ها بیشتر یا مساوی(اگر فقط همان دانه باشد)هزینه ی خوردن یک دانه است.
    <br/>
    Consistent بودن الگوریتم نیز به شکل زیر اثبات می شود:
    <br/>
    کمترین فاصله لازم برای رسیدن از F به H را با FH نشان می دهیم.باید ثابت کنیم اگر حالت بازی از 1 به 2 برود هزینه ی واقعی حرکت مار از 1 به 2 بیشتر یا مساوی اختلاف heuristic این دو حالت است.
    <br/>
    در حالت یک داریم:
    <br/>
    <li>سر مار روی نقطه ی C است.</li>
    <li>نقطه ای که بیشترین فاصله از مار را دارد نقطه ی B است.</li>
    در حالت دو داریم:
    <li>سر مار روی نقطه ی 'C است.</li>
    <li>نقطه ای که بیشترین فاصله از مار را دارد نقطه ی  Aاست.</li>  
</div>
<div style="direction:ltr;font-size: 14px;text-align: center;line-height: 2">
    1. h(1) = CB
    <br/>
    2. h(2) = C'A
    <br/>
    3. h(2) - h(1) = C'A - CB , CA &leq; CB
    <br/>
    4. h(2) - h(1) = C'A - CB &leq; C'A - CA = C'C
  
</div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    در قسمت سه فرض CA &leq; CB نوشته شده است که بیان می کند در حالت 1،نقطه ی B کمترین فاصله از سر مار را دارد بنابراین فاصله ی هر نقطه ی دیگری تا سر مار کمتر یا مساوی CB است.
    <br/>
    قسمت چهار نشان می دهد که اختلاف دوheirustic کمتر از CC’ (کمترین فاصله بین دو سر مار)است.همچنین کمترین هزینه برای رفتن از حالت 1 به 2 کمترین تعداد حرکات برای تکان دادن سر مار و رسیدن به نقطه ی جدید است بنابراین الگوریتم consistent نیز هست.
</div>    
    


In [11]:
def calculate_second_heuristic(node,alpha):
    seeds_position = node.seeds.keys()
    max_distance = 0 
    snake_head = node.snake[0]
    for pos in seeds_position:
        if(pos[1]>snake_head[1]):
            x1 = pos[1]-snake_head[1]
            x2 = snake_head[1]+boundry[1]-pos[1]
        else:
            x1 = snake_head[1]-pos[1]
            x2 = pos[1]+boundry[1]-snake_head[1]
        x = min(x1,x2)
        if(pos[0]>snake_head[0]):
            y1 = pos[0]-snake_head[0]
            y2 = snake_head[0]+boundry[0]-pos[0]
        else:
            y1 = snake_head[0]-pos[0]
            y2 = pos[0]+boundry[0]-snake_head[0]
        y = min(y1,y2)
        if(x+y>max_distance):
            max_distance = x+y
    return alpha*(max_distance)

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    هر دو تابع heuristic پیاده سازی شده پارامتر alpha را به عنوان ورودی دریافت می کنند.برای *A معمولی این ضریب برابر یک و برای *Weighted A برابر وزن مورد نظر قرار می گیرد.
    <br/>
    در صورتی که الگوریتم consistent نباشد در جست و جوی گراف به مشکل می خوریم با این که دو heuristic نوشته شده هر دو consistentهستند اما الگوریتم به گونه ای نوشته شده تا در صورت جست و جوی گراف به مشکل خورده نشود به این منظور هر نود در صورتی به مجموعه frontier اضافه می شود که یا در explored نباشد یا اگر بود نود دیده شده در سطح کمتری باشد تا باعث کم شدن هزینه شود.
   <br/>
    همچنین پس از اضافه کردن ضریب به heuristic، امکان دارد الگوریتم دیگر admissible نباشد.بنابراین روش *Wighted A لزوما جواب بهینه را پیدا نمی کند اما از آن جا که در بعضی موقعیت ها احتیاج به داشتن جواب(نه لزوما بهینه) با سرعت بیشتر داریم از این الگوریتم استفاده می کنیم.
</div>

In [12]:
class Astart_search:
    
    def __init__(self,initial_state,heuristic_function,weight):
        self.initial_state = initial_state
        self.calculate_heuristic = heuristic_function
        self.weight = weight
        

    def start_Astart(self):
        node = self.initial_state
        total_state = 1
        total_distinct_state = 1
        if node.is_goal():
            return node.path,total_state,total_distinct_state 
        frontier = PriorityQueue()
        frontier.push(node,self.calculate_heuristic(node,self.weight))
        explored = {}
        explored[node.get_key()] = node.level
        while True:
            if frontier.is_empty():
                return "",total_state,total_distinct_state 
            node = frontier.pop()
            childs = node.get_childs()
            for child in childs:
                total_state+=1
                if(child.get_key() not in explored or explored[child.get_key()]>child.level):
                    if child.get_key() not in explored:
                        total_distinct_state+=1
                    if child.is_goal():
                        return child.path,total_state,total_distinct_state 
                    frontier.push(child,self.calculate_heuristic(child,self.weight)+child.level)
                    explored[child.get_key()] = child.level

In [13]:
print("A* first heuristic")
start_time = time.time()
path, total_state, total_distinct_state = Astart_search(initial_state,calculate_first_heuristic,1).start_Astart()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ", total_state)
print("Distinct Nodes : ", total_distinct_state) 
print("____________________________________________________________")

print("A* second heuristic")
start_time = time.time()
path, total_state, total_distinct_state = Astart_search(initial_state,calculate_second_heuristic,1).start_Astart()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ", total_state)
print("Distinct Nodes : ", total_distinct_state) 
print("____________________________________________________________")

print("Weighted A* first heuristic,a=2")
start_time = time.time()
path, total_state, total_distinct_state = Astart_search(initial_state,calculate_first_heuristic,2).start_Astart()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ", total_state)
print("Distinct Nodes : ", total_distinct_state) 
print("____________________________________________________________")

print("Weighted A* second heuristic,a=2")
start_time = time.time()
path, total_state, total_distinct_state = Astart_search(initial_state,calculate_second_heuristic,2).start_Astart()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ", total_state)
print("Distinct Nodes : ", total_distinct_state) 
print("____________________________________________________________")

print("Weighted A* first heuristic,a=5")
start_time = time.time()
path, total_state, total_distinct_state = Astart_search(initial_state,calculate_first_heuristic,5).start_Astart()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ", total_state)
print("Distinct Nodes : ", total_distinct_state)
print("____________________________________________________________")

print("Weighted A* second heuristic,a=5")
start_time = time.time()
path, total_state, total_distinct_state = Astart_search(initial_state,calculate_second_heuristic,5).start_Astart()
print("Solution's time : %s seconds" % (time.time() - start_time))
print("Solution's Path : ", path)
print("Solution's depth : ", len(path))
print("Nodes  : ", total_state)
print("Distinct Nodes : ", total_distinct_state) 

A* first heuristic
Solution's time : 3.665807008743286 seconds
Solution's Path :  URRDDDRRDRRRDDRRULLDLUULL
Solution's depth :  25
Nodes  :  258368
Distinct Nodes :  125229
____________________________________________________________
A* second heuristic
Solution's time : 1.5820071697235107 seconds
Solution's Path :  URDDDRRDRRRRDDRRULLDLUULL
Solution's depth :  25
Nodes  :  92784
Distinct Nodes :  45516
____________________________________________________________
Weighted A* first heuristic,a=2
Solution's time : 1.3428478240966797 seconds
Solution's Path :  URRDDDRRDRRRDDRRULLDLUULL
Solution's depth :  25
Nodes  :  90027
Distinct Nodes :  49707
____________________________________________________________
Weighted A* second heuristic,a=2
Solution's time : 0.37921619415283203 seconds
Solution's Path :  URDDDRRRDLDRURRDDRURRDLDLU
Solution's depth :  26
Nodes  :  23528
Distinct Nodes :  12101
____________________________________________________________
Weighted A* first heuristic,a=5
Solut

<div style="direction:rtl;font-size:18px;"><b> مقایسه نتایج</b></div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    تابع create_results با گرفتن فایل ورودی و ایجاد استیت اولیه، همه ی الگوریتم های بالا را اجرا می کند و نتایج را در قالب یک dataframe  بر می گرداند. پس از دادن فایل های تست نتایج زیر به دست آمده است که در ادامه راجع به آن ها بحث شده است.
   </div>

In [14]:
def create_results(file_path):
    result = {}
    file = open(file_path, 'r') 
    lines = file.readlines() 
    count=0
    seeds = {}  
    for line in lines: 
        l = line.replace('\n','').split(",")
        if(count == 0):
            boundry = (int(l[0]),int(l[1]))
        if(count == 1):
            snake = [(int(l[0]),int(l[1]))]
        if(count>2):
            seeds[(int(l[0]),int(l[1]))] = int(l[2]) 
        count+=1
    initial_state = Node(snake,seeds,boundry)
    
    times = [0]*8
    path = [None]*8
    total_state = [0]*8
    total_distinct_state = [0]*8
    
    for i in range(0,2):
        #BFS
        start_time = time.time()
        path[0], total_state[0], total_distinct_state[0] = BFS_search(initial_state).start_BFS()
        end_time = time.time()
        times[0] += end_time-start_time
        #IDS
        start_time = time.time()
        path[1], total_state[1], total_distinct_state[1] = IDS_search(initial_state).start_IDS()
        end_time = time.time()
        times[1] += end_time-start_time
        #A*(1)
        start_time = time.time()
        path[2], total_state[2], total_distinct_state[2] = Astart_search(initial_state,calculate_first_heuristic,1).start_Astart()
        end_time = time.time()
        times[2] += end_time-start_time
        #A*(2)
        start_time = time.time()
        path[3], total_state[3], total_distinct_state[3] = Astart_search(initial_state,calculate_second_heuristic,1).start_Astart()
        end_time = time.time()
        times[3] += end_time-start_time
        #A*(1),a=2
        start_time = time.time()
        path[4], total_state[4], total_distinct_state[4] = Astart_search(initial_state,calculate_first_heuristic,2).start_Astart()
        end_time = time.time()
        times[4] += end_time-start_time
        #A*(2),a=2
        start_time = time.time()
        path[5], total_state[5], total_distinct_state[5] = Astart_search(initial_state,calculate_second_heuristic,2).start_Astart()
        end_time = time.time()
        times[5] += end_time-start_time
        #A*(1),a=5
        start_time = time.time()
        path[6], total_state[6], total_distinct_state[6] = Astart_search(initial_state,calculate_first_heuristic,5).start_Astart()
        end_time = time.time()
        times[6] += end_time-start_time
        #A*(2),a=5
        start_time = time.time()
        path[7], total_state[7], total_distinct_state[7] = Astart_search(initial_state,calculate_second_heuristic,5).start_Astart()
        end_time = time.time()
        times[7] += end_time-start_time
    labels = ["BFS","IDS","A*(1)","A*(2)","A*(1),a=2","A*(2),a=2","A*(1),a=5","A*(2),a=5"]
    for i,label in enumerate(labels):
        result[label] = {"distance":len(path[i]),"path":path[i],"total_state":total_state[i],"total_distinct_state":total_distinct_state[i],"time":times[i]/2}
    result_df = pd.DataFrame(data=result).T
    styles = [{'selector':'th','props':[('text-align','center')]},
              {'selector':'caption','props':[('text-align','center'),('font-size', '14px')]}]
    result_df = result_df.style.set_properties(**{'text-align': 'center'}).set_table_styles(styles)
    return result_df

In [15]:
test1 = create_results('./tests/test1.txt')
test1.set_caption("TEST1")
display(test1)

,distance,path,total_state,total_distinct_state,time
BFS,12,DLUULULLUULL,9595,4592,0.076332
IDS,12,LDLDRULUULUL,39422,4508,0.450334
A*(1),12,DLUULULLUULL,4836,2935,0.056899
A*(2),12,DLDLURUULULL,2517,1478,0.033526
"A*(1),a=2",12,DLUULULLUULL,1238,1015,0.015333
"A*(2),a=2",13,DLDLURDDLDLUL,2031,1198,0.026883
"A*(1),a=5",12,DLUULULLUULL,91,86,0.001164
"A*(2),a=5",16,RDDDRRDDRDDLUURD,740,438,0.009854


In [16]:
test2 = create_results('./tests/test2.txt')
test2.set_caption("TEST2")
display(test2)

,distance,path,total_state,total_distinct_state,time
BFS,15,URDLLUUUULULLLL,100566,45403,0.954916
IDS,15,LRRULLLUUULLLLU,522529,45599,6.823910
A*(1),15,RULLDLUUUUULLLL,72389,35105,1.014660
A*(2),15,RLLURUUULLULLLL,16510,9516,0.292299
"A*(1),a=2",15,RULLDLUUUUULLLL,29185,16804,0.429170
"A*(2),a=2",15,RLLURUUULLULLLL,11217,6353,0.190078
"A*(1),a=5",17,ULDRRUUUULLLULLLL,3312,2360,0.045908
"A*(2),a=5",17,RLLURUUUULLDLULLL,106210,38105,1.561205


In [17]:
test3 = create_results('./tests/test3.txt')
test3.set_caption("TEST3")
display(test3)

,distance,path,total_state,total_distinct_state,time
BFS,25,URRDDDRRDRRRDDRRULLDLUULL,434506,195025,4.252705
IDS,25,RUDDDRDRRDDRRRRRULLDLLLUU,3831299,216618,50.336221
A*(1),25,URRDDDRRDRRRDDRRULLDLUULL,258368,125229,3.848752
A*(2),25,URDDDRRDRRRRDDRRULLDLUULL,92784,45516,1.513752
"A*(1),a=2",25,URRDDDRRDRRRDDRRULLDLUULL,90027,49707,1.371439
"A*(2),a=2",26,URDDDRRRDLDRURRDDRURRDLDLU,23528,12101,0.357468
"A*(1),a=5",26,URRDDDRRDRULDDRRRDRURULLDD,919,740,0.013351
"A*(2),a=5",26,URDDDRRDRDLURRRDDRURRDLDLU,12698,5810,0.207010


<div style="direction:rtl;font-size:17px;"><li><b>زمان</b></li></div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    همان طور که دیده می شود الگوریتم IDS در هر سه تست زمان بیشتری برده است اما این الگوریتم به طور کلی حافظه ی مصرفی کمتری دارد بنابراین در جایی که محدودیت حافظه مهم تر از زمان پیدا کردن جواب است از این الگوریتم استفاده می شود.همچنین می توان دید هر چه جواب در عمق بیشتری باشد تفاوت زمان اجرای آن به نسبت الگوریتم های دیگر بیشتر است.
    <br/>
    BFS در جایگاه دوم قرار می گیرد چرا که اولویت خاصی به نود ها نمی دهد و همه ی آن ها را بر اساس سطح، جست و جو می کند.
    <br/>
    بهترین زمان برای الگوریتم *A است چرا که با در نظر گرفتن heuristic برای نود ها آن ها را در جهت نود هدف گسترش می دهد بنابراین سریع تر به جواب می رسد.
    <br/>
    بسته به تخمین زده شده *Weighted A می تواند زمان بهتر یا بدتری نسبت به *A بدهد برای مثال در تست یک از آن جا که heuristic خیلی ساده ای داشت احتمالا مقادیر تخمین زده شده برای آن خیلی کم تر بود و ضریب 2,5 هر دو باعث شدند که هم جواب بهینه و هم در کوتاه ترین زمان پاسخ به دست آید اما در تست دو *Weighted A با ضریب 5 در heuristic دوم زمان بیشتری نسبت به *A دارد.اما به طور میانگین می توان دید که *Weighted A زمان های کمتری دارد.
    
</div>    

<div style="direction:rtl;font-size:17px;"><li><b>استیت های مجزای دیده شده</b></li></div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
    به طور میانگین می توان گفت تعداد استیت های مجزای دیده شده در IDS بیشتر از BFS و در *A از همه کم تر است.در *Weighted A نیز با دادن ضریب تعداد استیت ها را کمتر می کند.هرچه هیوریستک نا دقیق تر باشد بزرگ تر کردن ضریب تا جای مشخصی تعداد استیت های مجزای دیده شده را کاهش می دهد.
</div>

<div style="direction:rtl;font-size:17px;"><li><b>استیت های دیده شده</b></li></div>
<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
استیت های دیده شده نیز از ترتیب گفته شده در قسمت قبل پیروی می کنند.نکته ی قابل توجه این است که در IDS تفاوت استیت های تکراری و غیر تکراری بسیار زیاد است و هر چه به سمت الگوریتم های *A می رویم این تفاوت کمتر می شود.
</div>

<div style="direction:rtl;font-size:17px;"><li><b>ضرایب *Weighted A</b></li></div>

<div style="direction:rtl;font-size: 16px;text-align: justify;line-height: 1.6">
  تابع heuristic اول تابع ساده ای بود و احتمالا مقدار را خیلی کمتر از مقدار واقعی تخمین میزد به همین علت هر دو ضرایب 2و5 باعث تولید جواب بهینه در تست یک شدند اما از آن جا که heuristic دوم به جواب کمی نزدیک تر بود ضریب  2باعث شد طول جواب یکی بیشتر از مقدار بهینه شود همچنین ضریب 5 نیز باعث دور تر شدن جواب از حالت بهینه شد.
    <br/>
    در تست دو ضرایب داده شده به heuristic ها تقریبا مشابه هم عمل کردند.با ضریب 2 هر دو جواب بهینه را تولید کردند و با ضریب 5 هر دو با جواب بهینه دو قدم فاصله داشتند.
    <br/>
    در آخرین تست نیز heuristic اول با ضریب 2 جواب بهینه را تولید کرد اما با افزایش ضریب مقادیر از مقدار واقعی دور شدند و جواب بهینه تولید نشد.
    <br/>
    اگر هر سه تست را مقایسه کنیم متوجه می شویم heuristic اول با ضریب 5 به مقدار قابل توجهی تعداد استیت های دیده شده را کاهش داده است.همچنین این heuristic با ضریب 2 به ازای همه ی تست ها پاسخ بهینه را پیدا کرده است.می توان گفت اگرچه این heuristic بدون ضریب ضعیف تر از heuristic دوم عمل کرده است ولی عملکرد آن با ضریب از heuristic دوم نیز بهتر بوده است.
</div>    